In [1]:
import zipfile36 as zipfile
import requests
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from io import BytesIO
import datetime as dt
import os
import pandas as pd
import numpy as np


In [2]:
header = {
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/111.0.0.0 Safari/537.36",
    "Sec-Fetch-User": "?1", "Accept": "*/*", "Sec-Fetch-Site": "none", "Sec-Fetch-Mode": "navigate",
    "Accept-Encoding": "gzip, deflate, br", "Accept-Language": "en-US,en;q=0.9,hi;q=0.8"
    }

In [3]:
def nse_urlfetch(url):
    r_session = requests.session()
    nse_live = r_session.get("http://nseindia.com", headers=header)
    return r_session.get(url, headers=header)

In [4]:
def fno_bhav_copy(trade_date: str):
    """
    new CM-UDiFF Common NSE future option bhav copy from 2018 on wards
    :param trade_date: eg:'01-06-2023'
    :return: pandas Data frame
    """
    # trade_date = datetime.strptime(trade_date, dd_mm_yyyy)
    trade_date = datetime.strptime(trade_date, "%d-%m-%Y")
    url = 'https://nsearchives.nseindia.com/content/fo/BhavCopy_NSE_FO_0_0_0_'
    payload = f"{str(trade_date.strftime('%Y%m%d'))}_F_0000.csv.zip"
    request_bhav = nse_urlfetch(url + payload)
    bhav_df = pd.DataFrame()
    if request_bhav.status_code == 200:
        zip_bhav = zipfile.ZipFile(BytesIO(request_bhav.content), 'r')
        for file_name in zip_bhav.filelist:
            if file_name:
                bhav_df = pd.read_csv(zip_bhav.open(file_name))
    elif request_bhav.status_code == 403:
        url2 = "https://www.nseindia.com/api/reports?archives=" \
             "%5B%7B%22name%22%3A%22F%26O%20-%20Bhavcopy(csv)%22%2C%22type%22%3A%22archives%22%2C%22category%22" \
             f"%3A%22derivatives%22%2C%22section%22%3A%22equity%22%7D%5D&date={str(trade_date.strftime('%d-%b-%Y'))}" \
             f"&type=equity&mode=single"
        request_bhav = nse_urlfetch(url2)
        if request_bhav.status_code == 200:
            zip_bhav = zipfile.ZipFile(BytesIO(request_bhav.content), 'r')
            for file_name in zip_bhav.filelist:
                if file_name:
                    bhav_df = pd.read_csv(zip_bhav.open(file_name))
        elif request_bhav.status_code == 403:
            raise FileNotFoundError(f' Data not found, change the date...')
    # bhav_df = bhav_df[['INSTRUMENT', 'SYMBOL', 'EXPIRY_DT', 'STRIKE_PR', 'OPTION_TYP', 'OPEN', 'HIGH', 'LOW',
    #                    'CLOSE', 'SETTLE_PR', 'CONTRACTS', 'VAL_INLAKH', 'OPEN_INT', 'CHG_IN_OI', 'TIMESTAMP']]
    return bhav_df



In [78]:
# curr_date = "02-01-2024"
curr_date = new_date_format
bhawcopy_df = fno_bhav_copy(curr_date)

In [79]:
date_obj = datetime.strptime(curr_date, "%d-%m-%Y")
# Format to YYYY-MM-DD
new_date_str = date_obj.strftime("%Y-%m-%d")
path = r"E:\\Automatic_Bhawcopy_Download\\sample_data_optput\\"
filename = path + new_date_str + "_bhawcopy.csv"
bhawcopy_df.to_csv(filename, index=False)

#### Create a Date Range for continious bhawcopy download

In [57]:
import holidays
start_date = "01-01-2024"
end_date = "08-09-2024"
# start_date and end date format %m-%d-%Y or %m/%d/%Y
# output format: 2024-01-05 00:00:00 in datetime format
date_range = pd.bdate_range(start = start_date, end = end_date, freq ='C',holidays = holidays.holidays(2024))

In [82]:
for index, cday in enumerate(date_range):
    # print(cday)
    temp_date = cday.date()
    curr_date = temp_date.strftime("%d-%m-%Y")
    print(curr_date, index)

    bhawcopy_df = fno_bhav_copy(curr_date)
    
    date_obj = datetime.strptime(curr_date, "%d-%m-%Y")
    # Format to YYYY-MM-DD
    new_date_str = date_obj.strftime("%Y-%m-%d")
    path = r"E:\Bhawcopy_2024\\"
    filename = path + new_date_str + "_bhawcopy.csv"
    bhawcopy_df.to_csv(filename, index=False)
    # break

01-01-2024 0
02-01-2024 1
03-01-2024 2
04-01-2024 3
05-01-2024 4
08-01-2024 5
09-01-2024 6
10-01-2024 7
11-01-2024 8
12-01-2024 9
15-01-2024 10
16-01-2024 11
17-01-2024 12
18-01-2024 13
19-01-2024 14
22-01-2024 15
23-01-2024 16
24-01-2024 17
25-01-2024 18
26-01-2024 19
29-01-2024 20
30-01-2024 21
31-01-2024 22
01-02-2024 23
02-02-2024 24
05-02-2024 25
06-02-2024 26
07-02-2024 27
08-02-2024 28
09-02-2024 29
12-02-2024 30
13-02-2024 31
14-02-2024 32
15-02-2024 33
16-02-2024 34
19-02-2024 35
20-02-2024 36
21-02-2024 37
22-02-2024 38
23-02-2024 39
26-02-2024 40
27-02-2024 41
28-02-2024 42
29-02-2024 43
01-03-2024 44
04-03-2024 45
05-03-2024 46
06-03-2024 47
07-03-2024 48
08-03-2024 49
11-03-2024 50
12-03-2024 51
13-03-2024 52
14-03-2024 53
15-03-2024 54
18-03-2024 55
19-03-2024 56
20-03-2024 57
21-03-2024 58
22-03-2024 59
25-03-2024 60
26-03-2024 61
27-03-2024 62
28-03-2024 63
29-03-2024 64
01-04-2024 65
02-04-2024 66
03-04-2024 67
04-04-2024 68
05-04-2024 69
08-04-2024 70
09-04-2024 71
10

In [77]:
new_date_format

'05-01-2024'

In [72]:
str(cday.date())

'2024-01-01'

In [73]:
temp_date = cday.date()
temp_date.strftime("%d-%m-%Y")

'01-01-2024'

#### TO Save CSV File Directlly

In [31]:
trade_date = "04-02-2021"
# trade_date = datetime.strptime(trade_date, dd_mm_yyyy)
trade_date = datetime.strptime(trade_date, "%d-%m-%Y")
url = 'https://nsearchives.nseindia.com/content/fo/BhavCopy_NSE_FO_0_0_0_'
payload = f"{str(trade_date.strftime('%Y%m%d'))}_F_0000.csv.zip"
request_bhav = nse_urlfetch(url + payload)
bhav_df = pd.DataFrame()
if request_bhav.status_code == 200:
    print(request_bhav)
    print(request_bhav.text)
    zip_bhav = zipfile.ZipFile(BytesIO(request_bhav.content), 'r')
    # print(zip_bhav)
    # zip_bhav = zipfile.ZipFile(BytesIO(request_bhav.content), 'r')
    # for file_name in zip_bhav.filelist:
    #     if file_name:
            # bhav_df = pd.read_csv(zip_bhav.open(file_name))
elif request_bhav.status_code == 403:
    url2 = "https://www.nseindia.com/api/reports?archives=" \
            "%5B%7B%22name%22%3A%22F%26O%20-%20Bhavcopy(csv)%22%2C%22type%22%3A%22archives%22%2C%22category%22" \
            f"%3A%22derivatives%22%2C%22section%22%3A%22equity%22%7D%5D&date={str(trade_date.strftime('%d-%b-%Y'))}" \
            f"&type=equity&mode=single"
    request_bhav = nse_urlfetch(url2)
    if request_bhav.status_code == 200:
        print(request_bhav)
        print(request_bhav.text)
        zip_bhav = zipfile.ZipFile(BytesIO(request_bhav.content), 'r')
        # print(zip_bhav)
        # zip_bhav = zipfile.ZipFile(BytesIO(request_bhav.content), 'r')
        # for file_name in zip_bhav.filelist:
        #     if file_name:
        #         bhav_df = pd.read_csv(zip_bhav.open(file_name))
    elif request_bhav.status_code == 403:
        raise FileNotFoundError(f' Data not found, change the date...')
# bhav_df = bhav_df[['INSTRUMENT', 'SYMBOL', 'EXPIRY_DT', 'STRIKE_PR', 'OPTION_TYP', 'OPEN', 'HIGH', 'LOW',
#                    'CLOSE', 'SETTLE_PR', 'CONTRACTS', 'VAL_INLAKH', 'OPEN_INT', 'CHG_IN_OI', 'TIMESTAMP']]



In [17]:
def fno_bhav_copy(trade_date: str):
    """
    new CM-UDiFF Common NSE future option bhav copy from 2018 on wards
    :param trade_date: eg:'01-06-2023'
    :return: pandas Data frame
    """
    # trade_date = datetime.strptime(trade_date, dd_mm_yyyy)
    trade_date = datetime.strptime(trade_date, "%d-%m-%Y")
    url = 'https://nsearchives.nseindia.com/content/fo/BhavCopy_NSE_FO_0_0_0_'
    payload = f"{str(trade_date.strftime('%Y%m%d'))}_F_0000.csv.zip"
    request_bhav = nse_urlfetch(url + payload)
    bhav_df = pd.DataFrame()
    if request_bhav.status_code == 200:
        zip_bhav = zipfile.ZipFile(BytesIO(request_bhav.content), 'r')
        for file_name in zip_bhav.filelist:
            if file_name:
                bhav_df = pd.read_csv(zip_bhav.open(file_name))
    elif request_bhav.status_code == 403:
        url2 = "https://www.nseindia.com/api/reports?archives=" \
             "%5B%7B%22name%22%3A%22F%26O%20-%20Bhavcopy(csv)%22%2C%22type%22%3A%22archives%22%2C%22category%22" \
             f"%3A%22derivatives%22%2C%22section%22%3A%22equity%22%7D%5D&date={str(trade_date.strftime('%d-%b-%Y'))}" \
             f"&type=equity&mode=single"
        request_bhav = nse_urlfetch(url2)
        if request_bhav.status_code == 200:
            zip_bhav = zipfile.ZipFile(BytesIO(request_bhav.content), 'r')
            for file_name in zip_bhav.filelist:
                if file_name.filename.endswith('.csv'):
                # Define the full path for saving the CSV file
                    output_directory = "E:\Automatic_Bhawcopy_Download\sample_data_and_O"
                    output_path = os.path.join(output_directory, file_name.filename)
                    
                    # Read the file directly from the ZIP archive and save it
                    with zip_bhav.open(file_name.filename) as file:
                        with open(output_path, 'wb') as output_file:
                            output_file.write(file.read())
        elif request_bhav.status_code == 403:
            raise FileNotFoundError(f' Data not found, change the date...')
    # bhav_df = bhav_df[['INSTRUMENT', 'SYMBOL', 'EXPIRY_DT', 'STRIKE_PR', 'OPTION_TYP', 'OPEN', 'HIGH', 'LOW',
    #                    'CLOSE', 'SETTLE_PR', 'CONTRACTS', 'VAL_INLAKH', 'OPEN_INT', 'CHG_IN_OI', 'TIMESTAMP']]
    return [os.path.join(output_directory, file_info.filename) for file_info in zip_bhav.filelist if file_info.filename.endswith('.csv')]




<>:29: SyntaxWarning: invalid escape sequence '\A'
<>:29: SyntaxWarning: invalid escape sequence '\A'
C:\Users\RAJU SINGH\AppData\Local\Temp\ipykernel_14536\818893707.py:29: SyntaxWarning: invalid escape sequence '\A'
  output_directory = "E:\Automatic_Bhawcopy_Download\sample_data_and_O"


In [18]:
fno_bhav_copy("09-08-2024")

C:\Users\RAJU SINGH\AppData\Local\Temp\ipykernel_14536\818893707.py:29: SyntaxWarning: invalid escape sequence '\A'
  output_directory = "E:\Automatic_Bhawcopy_Download\sample_data_and_O"


UnboundLocalError: cannot access local variable 'output_directory' where it is not associated with a value

In [8]:
!pip install pandas


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [84]:
start_date = "01-01-2024"
end_date = "28-01-2024"
# start_date and end date format %m-%d-%Y or %m/%d/%Y
# output format: 2024-01-05 00:00:00 in datetime format
date_range = pd.bdate_range(start = start_date, end = end_date, freq ='C',holidays = holidays.holidays(2024))
date_range

DatetimeIndex(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04',
               '2024-01-05', '2024-01-08', '2024-01-09', '2024-01-10',
               '2024-01-11', '2024-01-12', '2024-01-15', '2024-01-16',
               '2024-01-17', '2024-01-18', '2024-01-19', '2024-01-22',
               '2024-01-23', '2024-01-24', '2024-01-25', '2024-01-26'],
              dtype='datetime64[ns]', freq='C')

In [96]:
from holidays import holidays
holidays(2024)

[]

In [99]:
import holidays
holidays.holidays(2024)

[]